In [1]:
! pip install cma

     |████████████████████████████████| 230 kB 155 kB/s eta 0:00:01


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import timeit

import numpy as np
import cma
from IPython.display import display, clear_output

from anguilla.optimizers.cma import CMA, StrategyParameters, StoppingConditions
from anguilla.fitness.benchmark import Sphere, SumSquares

In [7]:
def test_with_pycma(n=4):
    fitness_function = Sphere()
    initial_point = fitness_function.propose_initial_point(n)
    initial_sigma = 0.3
    cma.fmin(fitness_function, initial_point, initial_sigma, options={'CMA_Active': 0})

test_with_pycma()

(4_w,8)-CMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=720738, Sun Nov 29 00:27:55 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 7.790223198024215e-01 1.0e+00 2.83e-01  3e-01  3e-01 0:00.0
    2     16 3.845624480190633e-01 1.1e+00 2.91e-01  3e-01  3e-01 0:00.0
    3     24 2.784872783050687e-01 1.2e+00 3.15e-01  3e-01  3e-01 0:00.0
  100    800 1.035019245906245e-13 1.6e+00 1.08e-05  3e-07  4e-07 0:00.3
  112    896 3.267393054913799e-15 1.4e+00 1.31e-06  2e-08  2e-08 0:00.3
termination on tolfun=1e-11 (Sun Nov 29 00:27:59 2020)
final/bestever f-value = 3.100885e-15 2.113454e-15
incumbent solution: [5.123875486645053e-09, -5.2446199674011465e-08, 3.434494610055259e-10, -1.799748709826705e-08]
std deviation: [2.1148474246063247e-08, 2.150244810065341e-08, 2.059083318946686e-08, 1.869662470600328e-08]


In [9]:
def test_with_prototype(n=4):
    parameters = StrategyParameters(n=n)
    conditions = StoppingConditions(n=n,
                                    population_size=parameters.population_size,
                                    ftarget=1e-14,
                                    )
    fitness_function = Sphere()
    initial_point = fitness_function.propose_initial_point(n)
    optimizer = CMA(initial_point=initial_point,
                    initial_sigma=0.3,
                    strategy_parameters=parameters,
                    stopping_conditions=conditions)
    while not optimizer.stop():
        candidate_solutions = optimizer.ask()
        function_values = fitness_function(candidate_solutions.T)
        ranked_indices = optimizer.rank(candidate_solutions, function_values)
        optimizer.tell(candidate_solutions, ranked_indices)
        if (optimizer._fevals % 20) == 0:
            display(optimizer._fevals, optimizer._best_value, optimizer._best_solution)
    display(optimizer._fevals, optimizer._best_value, optimizer._best_solution)

test_with_prototype(n=4)

20

6.985819637441909

array([ 0.45917604, -0.14509201,  0.56417126, -2.53685555])

40

3.41088752353529

...

3.1985533470160406e-14

array([-1.23101027e-07, -3.94755193e-08,  7.24712616e-08, -1.00106295e-07])

845

2.2673874667687895e-15

array([-2.29839331e-08,  3.44687446e-08,  2.31646493e-08,  3.79880982e-09])